- json 개수 959개 확인
- road 개수 959개 확인
- camera 개수 959개 확인

In [8]:
new_path = 'G:/ADIN22023_AIMMO_AD/2_PJT/5_Data_Refinement/2_post/2_FromEng/etc%2E/TestCase Opendata GT/20230331/오픈데이터셋'

In [10]:
import glob

new_list = glob.glob(os.path.join(new_path,'**/*.json'),recursive=True)


In [14]:
len(new_list)

549

In [1]:
import json
import os
import re
from tqdm import tqdm

except_case = {'update_error':[],'del_error':[]}
original_path = 'D:/data/testcase/Typical_case_data/rev2_edit1/label_bb'
move_path = 'C:/Users/aimmo-aiy-0169/Desktop/azk'
camera_path = "D:/data/testcase/Typical_case_data/rev2_edit1/new_retouching/TestCase_리터칭 2차 (카메라)"
road_path = "D:/data/testcase/Typical_case_data/rev2_edit1/new_retouching/TestCase 리터칭 2차 (도로 정보)"

update_key_roots = [road_path,camera_path]

# 전체 JSON file을 탐색하고 제너레이터로 1개씩 return
def find_json_files_in_directory(original_path):
    for root,dirs,files in os.walk(original_path):
        for file in files :
            if file.endswith('.json'):
                yield os.path.join(root,file)
                
# 이동 경로에 기존 경로와 동일한 폴더 구조들을 생성한다. -> 전체 json file들 구조
def create_directory_structure(original_path,move_path):
    for root,dirs,files in os.walk(original_path):
        new_path = os.path.join(move_path, os.path.relpath(root, original_path))
        os.makedirs(new_path,exist_ok=True)

        
# 옮길 json 파일의 폴더 구조와 동일한 구조를 이동 폴더에 생성 -> 단일 json file 구조        
def create_directory(json_file_path,original_path,move_path):
    new_path = os.path.dirname(json_file_path.replace(original_path,move_path))
    os.makedirs(new_path,exist_ok=True)

    
# json file에서 key값들 삭제    
def delete_keys_from_json(json_data,delete_keys = ['road_type',"illumination_status","road_status","sensor_status"]):
    try :
        for key in delete_keys:
            del json_data[key]
    except :
        except_case['del_error'].append(json_data['parent_path'] + '/' +json_data['filename'])
    return json_data

# json file에서 key값들 추가
def update_keys_from_json(json_data,json_file_path,original_path):
    add_keys = []
    new_dict = {}
    for update_root in update_key_roots:
        path = json_file_path.replace(original_path,update_root)
        try :
            with open(path,'r',encoding='utf-8') as f:
                update_json = json.load(f)
                add_keys.append(update_json['attributes'])
        except :
            except_case['update_error'].append( path)
       
    for key,value in json_data.items():
        new_dict[key] = value
        if key == 'road_feature':
            for add_key,add_value in add_keys[0].items():
                new_dict[add_key] = add_value

        elif key == 'season':
            for add_key,add_value in add_keys[1].items():
                new_dict[add_key] = add_value
    return new_dict

# json file 수정
def change_keys_from_json(original_path,move_path):
    # 생성경로 directory 생성
    create_directory_structure(original_path,move_path)
    
    # json 읽기
    for json_file_path in tqdm(find_json_files_in_directory(original_path)):
        with open(json_file_path,'r',encoding='utf-8') as f:
            json_data = json.load(f)
            
            # 특정 key 삭제
            json_data = delete_keys_from_json(json_data)
                
                
            # 특정 key 추가
            json_file_path=json_file_path.replace('_Bbox_GT','.png')
            json_data = update_keys_from_json(json_data,json_file_path,original_path)
            
        # json 저장
        new_path = os.path.join(move_path, os.path.relpath(json_file_path,original_path))
        with open(new_path, 'w',encoding='utf-8') as f:
            json.dump(json_data, f,ensure_ascii=False, indent=4)

In [ ]:
change_keys_from_json(original_path,move_path)

In [13]:
print(except_case['del_error'])
len(except_case['del_error'])

[]


0

In [533]:
# 에러 저장
with open('del_error.txt', 'w') as f:
    for item in except_case['del_error']:
        f.write("%s\n" % item)

In [ ]:
import os
import json

# Traverse the original directory recursively to find all JSON files
def find_json_files(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.json'):
                yield os.path.join(root, file)

# Create a new directory structure for the JSON files
def create_new_directory_structure(directory, new_directory):
    for root, dirs, files in os.walk(directory):
        for dirr in dirs:
            path = os.path.join(new_directory, os.path.relpath(os.path.join(root, dirr), directory))
            os.makedirs(path, exist_ok=True)

# Copy the JSON files to the new directory structure
def copy_json_files(directory, new_directory):
    for json_file in find_json_files(directory):
        with open(json_file, 'r') as f:
            json_data = json.load(f)
        
        new_path = os.path.join(new_directory, os.path.relpath(json_file, directory))
        os.makedirs(os.path.dirname(new_path), exist_ok=True)
        
        with open(new_path, 'w') as f:
            json.dump(json_data, f, indent=4)

# Remove empty directories in the new directory structure
def remove_empty_directories(directory):
    for root, dirs, files in os.walk(directory, topdown=False):
        for dir in dirs:
            path = os.path.join(root, dir)
            if not os.listdir(path):
                os.rmdir(path)

# Example usage
directory = 'path/to/directory'
new_directory = 'path/to/new/directory'

create_new_directory_structure(directory, new_directory)
copy_json_files(directory, new_directory)
remove_empty_directories(new_directory)
